<a href="https://colab.research.google.com/github/romapavelko01/NLP_SDLC_project/blob/classifications/initial_text_classification_by_SHDESCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Reading a json-file

In [3]:
df = pd.read_json('/content/drive/MyDrive/SDLC/news_analysis_project/data/final_news_category_dataset.json', orient='split')
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [4]:
# import nltk
# nltk.download('stopwords')
# stop_words = set(stopwords.words('english'))
df['sentences_count'] = df['short_description'].apply(lambda x: len(x.split('.')))
df['sentences_count'].mean()

2.1187186648942262

In [5]:
df['date'].quantile([0, 1])

0.0   2012-01-28
1.0   2018-05-26
Name: date, dtype: datetime64[ns]

In [6]:
len(df)

200853

Giving specific numbers for each distinct category of the newly formed smaller dataframe

In [7]:
small_df = df[['category', 'short_description']]
small_df['category_id'] = small_df['category'].factorize()[0]
small_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,category,short_description,category_id
0,CRIME,She left her husband. He killed their children...,0
1,ENTERTAINMENT,Of course it has a song.,1
2,ENTERTAINMENT,The actor and his longtime girlfriend Anna Ebe...,1
3,ENTERTAINMENT,The actor gives Dems an ass-kicking for not fi...,1
4,ENTERTAINMENT,"The ""Dietland"" actress said using the bags is ...",1


Feature extraction using tfidf-vectorizer

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(small_df.short_description)
labels = small_df.category_id
features

<200853x54037 sparse matrix of type '<class 'numpy.float64'>'
	with 2198330 stored elements in Compressed Sparse Row format>

# Working with original dataset, no over/under-sampling

## Without considering stop_words explicitly

Building Multinomial Bayesian Classifier

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
X_train, X_test, y_train, y_test = train_test_split(small_df['short_description'], small_df['category'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [10]:
predicted_MB = clf.predict(count_vect.transform(X_test))
predicted_MB

array(['POLITICS', 'POLITICS', 'WELLNESS', ..., 'POLITICS', 'POLITICS',
       'WELLNESS'], dtype='<U14')

In [11]:
from sklearn.metrics import accuracy_score

In [12]:
acc_MNB_count = accuracy_score(y_test, predicted_MB)
acc_MNB_count

0.29372286613295095

The following code raises  NO. ITERATIONS LIMIT REACHED (to be worked on)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
  model_name = model.__class__.__name__
  accuracies = cross_val_score(model, features, labels, scoring='accuracy', cv=CV)
  for fold_idx, accuracy in enumerate(accuracies):
    entries.append((model_name, fold_idx, accuracy))
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
import seaborn as sns
sns.boxplot(x='model_name', y='accuracy', data=cv_df)
sns.stripplot(x='model_name', y='accuracy', data=cv_df, 
              size=8, jitter=True, edgecolor="gray", linewidth=2)
plt.show()

KeyboardInterrupt: ignored

Using Multinomial Naive Bayesian classifier (with pipeline)

In [13]:
from sklearn.pipeline import Pipeline

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.3044569243637233


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

          ARTS       0.00      0.00      0.00       367
ARTS & CULTURE       0.00      0.00      0.00       335
  BLACK VOICES       0.00      0.00      0.00      1170
      BUSINESS       0.00      0.00      0.00      1480
       COLLEGE       0.00      0.00      0.00       278
        COMEDY       0.50      0.00      0.00      1283
         CRIME       0.00      0.00      0.00       834
CULTURE & ARTS       0.00      0.00      0.00       238
       DIVORCE       1.00      0.01      0.01       847
     EDUCATION       0.00      0.00      0.00       250
 ENTERTAINMENT       0.43      0.28      0.34      3981
   ENVIRONMENT       0.00      0.00      0.00       318
         FIFTY       0.00      0.00      0.00       366
  FOOD & DRINK       0.60      0.14      0.23      1578
     GOOD NEWS       0.00      0.00      0.00       348
         GREEN       0.00      0.00      0.00       621
HEALTHY LIVING       0.00      0.00      0.00  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Working with Linear Support Vector Machine (using CountVectorizer)

In [14]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)


y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.42221292866531246
                precision    recall  f1-score   support

          ARTS       0.11      0.04      0.05       367
ARTS & CULTURE       0.16      0.04      0.06       335
  BLACK VOICES       0.34      0.16      0.22      1170
      BUSINESS       0.39      0.21      0.27      1480
       COLLEGE       0.22      0.12      0.16       278
        COMEDY       0.25      0.10      0.14      1283
         CRIME       0.38      0.24      0.29       834
CULTURE & ARTS       0.19      0.18      0.19       238
       DIVORCE       0.55      0.56      0.55       847
     EDUCATION       0.28      0.12      0.17       250
 ENTERTAINMENT       0.41      0.35      0.38      3981
   ENVIRONMENT       0.30      0.19      0.24       318
         FIFTY       0.08      0.04      0.05       366
  FOOD & DRINK       0.48      0.55      0.52      1578
     GOOD NEWS       0.08      0.04      0.05       348
         GREEN       0.27      0.10      0.15       621
HEALTHY LIVING    

Logistic Regression (gives ITERATIONS LIMIT REACHED)

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

NameError: ignored

## Working with a dataset with stopwords removed by setting parameter *stop_words='english'* inside the *CountVectorizer()*

Multinomial Naive Bayesian

In [15]:
from sklearn.pipeline import Pipeline

nb = Pipeline([('vect', CountVectorizer(stop_words='english')),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.3267813757119528


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                precision    recall  f1-score   support

          ARTS       0.00      0.00      0.00       367
ARTS & CULTURE       0.00      0.00      0.00       335
  BLACK VOICES       0.00      0.00      0.00      1170
      BUSINESS       0.80      0.00      0.01      1480
       COLLEGE       0.00      0.00      0.00       278
        COMEDY       0.50      0.00      0.00      1283
         CRIME       0.00      0.00      0.00       834
CULTURE & ARTS       0.00      0.00      0.00       238
       DIVORCE       0.94      0.04      0.07       847
     EDUCATION       0.00      0.00      0.00       250
 ENTERTAINMENT       0.39      0.34      0.36      3981
   ENVIRONMENT       0.00      0.00      0.00       318
         FIFTY       0.00      0.00      0.00       366
  FOOD & DRINK       0.62      0.27      0.37      1578
     GOOD NEWS       0.00      0.00      0.00       348
         GREEN       0.00      0.00      0.00       621
HEALTHY LIVING       1.00      0.00      0.01  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Linear Support Vector Machine

In [16]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(stop_words='english')),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)


y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.41990281594774365
                precision    recall  f1-score   support

          ARTS       0.13      0.05      0.07       367
ARTS & CULTURE       0.21      0.06      0.09       335
  BLACK VOICES       0.33      0.16      0.21      1170
      BUSINESS       0.38      0.21      0.27      1480
       COLLEGE       0.24      0.14      0.18       278
        COMEDY       0.26      0.10      0.14      1283
         CRIME       0.38      0.21      0.27       834
CULTURE & ARTS       0.27      0.18      0.21       238
       DIVORCE       0.56      0.57      0.57       847
     EDUCATION       0.36      0.16      0.22       250
 ENTERTAINMENT       0.42      0.34      0.37      3981
   ENVIRONMENT       0.32      0.16      0.21       318
         FIFTY       0.10      0.05      0.06       366
  FOOD & DRINK       0.47      0.56      0.51      1578
     GOOD NEWS       0.05      0.03      0.03       348
         GREEN       0.25      0.12      0.16       621
HEALTHY LIVING    

## Creating a new dataset with stopwords removed manually - checking presence of words in the nltk.stopwords list

In [17]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [18]:
import re
import string


def cleaning_function(sentence):
    """
    Function needed to perform data preprocessing: removing punctuation symbols,
    stop_words and other random things in order to obtain clean text
    """
    # the following line removes numbers from text
    result = re.sub(r'\d+', '', sentence.lower())

    # the following line removes any punctuation from the text
    result = result.translate(str.maketrans('','',string.punctuation))
    return [word for word in result.split() if not word in stop_words]


def collect_words(lst):
    """
    Given a list of lists of words,
    return a list of all the words in a list representation of the text.
    """
    return [word for sentence in lst for word in sentence]


In [19]:
new_df = small_df.copy()
new_df['sentences'] = new_df['short_description'].apply(lambda x: [i for i in x.split('.') if i])
new_df['filtered_sentences'] = new_df['sentences'].apply(lambda x: [cleaning_function(sentence) for sentence in x])
# instance = new_df['sentences'][0]
new_df['filtered_text'] = new_df['filtered_sentences'].apply(lambda x: ' '.join(collect_words(x)))
# x = [cleaning_function(i) for i in instance]
new_df = new_df[['category', 'filtered_text']]
new_df.head()

,category,filtered_text
0,CRIME,left husband killed children another day america
1,ENTERTAINMENT,course song
2,ENTERTAINMENT,actor longtime girlfriend anna eberstein tied ...
3,ENTERTAINMENT,actor gives dems asskicking fighting hard enou...
4,ENTERTAINMENT,dietland actress said using bags really cathar...


Splitting the dataset into train and test

In [20]:
X_train, X_test, y_train, y_test = train_test_split(new_df['filtered_text'], new_df['category'], random_state = 0)


### Training Multinomial Naive Bayseian classifier on the new data

In [21]:
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [22]:
predicted_MB = clf.predict(count_vect.transform(X_test))
predicted_MB

array(['POLITICS', 'POLITICS', 'WELLNESS', ..., 'WELLNESS', 'POLITICS',
       'WELLNESS'], dtype='<U14')

In [23]:
acc_MNB_count = accuracy_score(y_test, predicted_MB)
acc_MNB_count

0.34249412514438204

### Training Linear Support Vector Machine on the new data

In [24]:
sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)


y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.4177520213486279
                precision    recall  f1-score   support

          ARTS       0.14      0.04      0.07       367
ARTS & CULTURE       0.16      0.04      0.07       335
  BLACK VOICES       0.33      0.16      0.22      1170
      BUSINESS       0.39      0.22      0.28      1480
       COLLEGE       0.27      0.12      0.17       278
        COMEDY       0.23      0.08      0.12      1283
         CRIME       0.38      0.24      0.29       834
CULTURE & ARTS       0.25      0.18      0.21       238
       DIVORCE       0.55      0.56      0.55       847
     EDUCATION       0.33      0.15      0.21       250
 ENTERTAINMENT       0.25      0.53      0.34      3981
   ENVIRONMENT       0.34      0.16      0.22       318
         FIFTY       0.12      0.05      0.08       366
  FOOD & DRINK       0.48      0.56      0.51      1578
     GOOD NEWS       0.06      0.03      0.04       348
         GREEN       0.29      0.11      0.16       621
HEALTHY LIVING     

# Testing oversampling

In [26]:
count = y_train.value_counts()
n_samples = count.median()
n_samples

2571.0

In [27]:
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import SMOTE

In [28]:
textclassifier =Pipeline([
  ('vect', CountVectorizer()),
   ('tfidf', TfidfTransformer()),
   ('smote', SMOTE(random_state=12)),
   ('mnb', MultinomialNB(alpha =0.1))
])

Forming a smaller dataset to check how long it will take MB to fit 

In [29]:
half_df = new_df.sample(frac=0.5)
X_train, X_test, y_train, y_test = train_test_split(half_df['filtered_text'], half_df['category'], random_state = 0)


In [30]:
textclassifier.fit(X_train, y_train)

Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('smote', SMOTE(random_state=12)),
                ('mnb', MultinomialNB(alpha=0.1))])

In [33]:
X_test.shape

(25107,)

In [34]:
half_predicted =  textclassifier.predict(X_test)
accuracy_score(y_test, half_predicted)
# half_predicted

0.35129645118891145